In [1]:
%%capture
%load_ext autoreload
%autoreload 2
#Basic Imports
import os,sys
os.chdir("..")

from tqdm import tqdm,trange
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix, ConfusionMatrixDisplay
import torch
import pandas

from datasets.ssl_dataset import SSL_Dataset
from datasets.data_utils import get_data_loader
from utils import get_model_checkpoints
from utils import net_builder

In [2]:
#Path to the runs to load
csv_folder = "/home/gabrielemeoni/project/SSLRS/test"
folder = "/scratch/fixmatch_results/nr_of_labels_supervised/eurosat_rgb"

## Initialize parameters

In [3]:
checkpoints, run_args = get_model_checkpoints(folder)
[print(_.split("/")[1]) for _ in checkpoints];

scratch
scratch
scratch
scratch
scratch


## Run all models

In [4]:
results = []
for checkpoint, args in zip(checkpoints,run_args):
    print("------------ RUNNING ", checkpoint, " -----------------")
    print(args)
    args["batch_size"] = 256
    args["data_dir"] = "./data/"
    args["use_train_model"] = False
    args["load_path"] = checkpoint
    
    checkpoint_path = os.path.join(args["load_path"])
    checkpoint = torch.load(checkpoint_path,map_location='cuda:0')
    load_model = (checkpoint["train_model"] if args["use_train_model"] else checkpoint["eval_model"])

    _net_builder = net_builder(args["net"],False,{})

    net = _net_builder(num_classes=args["num_classes"])
    net.load_state_dict(load_model)
    if torch.cuda.is_available():
        net.cuda()
    net.eval()

    _eval_dset = SSL_Dataset(name=args["dataset"], train=False, data_dir=args["data_dir"], seed=args["seed"])
    eval_dset = _eval_dset.get_dset()

    eval_loader = get_data_loader(eval_dset, args["batch_size"], num_workers=1)
    label_encoding = _eval_dset.label_encoding
    inv_transf = _eval_dset.inv_transform

    
    print("------------ PREDICTING TESTSET -----------------")
    
    images, labels, preds = [],[],[]
    with torch.no_grad():
        for image, target in tqdm(eval_loader):
            image = image.type(torch.FloatTensor).cuda()
            logit = net(image)
            for idx,img in enumerate(image):
                images.append(inv_transf(img.transpose(0,2).cpu().numpy()).transpose(0,2).numpy())
            preds.append(logit.cpu().max(1)[1])
            labels.append(target)
    labels = torch.cat(labels).numpy()
    preds = torch.cat(preds).numpy()
    test_report = classification_report(labels, preds, target_names=label_encoding, output_dict=True)
    test_report["params"] = args
    results.append(test_report)

------------ RUNNING  /scratch/fixmatch_results/nr_of_labels_supervised/eurosat_rgb/FixMatch_archefficientnet-b2_batch32_confidence0.95_lr0.03_nclass10_uratio7_wd0.0001_wu0.0_seed0_numlabels100_optSGD/model_best.pth  -----------------
{'dataset': 'eurosat_rgb', 'net': 'efficientnet-b2', 'batch': 32, 'confidence': 0.95, 'lr': 0.03, 'num_classes': 10, 'uratio': 7, 'wd': 0.0001, 'wu': 0.0, 'seed': 0, 'numlabels': 100, 'opt': 'SGD', 'iterations': 496000}


  0%|          | 0/10 [00:00<?, ?it/s]

Using not pretrained model efficientnet-b2 ...


  0%|          | 0/11 [00:00<?, ?it/s]

------------ PREDICTING TESTSET -----------------


...

100%|██████████| 11/11 [00:01<00:00,  8.35it/s]


------------ RUNNING  /scratch/fixmatch_results/nr_of_labels_supervised/eurosat_rgb/FixMatch_archefficientnet-b2_batch32_confidence0.95_lr0.03_nclass10_uratio7_wd0.0001_wu0.0_seed0_numlabels500_optSGD/model_best.pth  -----------------
{'dataset': 'eurosat_rgb', 'net': 'efficientnet-b2', 'batch': 32, 'confidence': 0.95, 'lr': 0.03, 'num_classes': 10, 'uratio': 7, 'wd': 0.0001, 'wu': 0.0, 'seed': 0, 'numlabels': 500, 'opt': 'SGD', 'iterations': 27000}
Using not pretrained model efficientnet-b2 ...


  0%|          | 0/11 [00:00<?, ?it/s]

------------ PREDICTING TESTSET -----------------


100%|██████████| 11/11 [00:01<00:00,  8.02it/s]


In [5]:
import pandas as pd
big_df = pd.DataFrame()
pd.set_option('display.max_columns', None)
for result in results:
    params = result["params"]
    df = pd.DataFrame(result)
    df.drop(list(params.keys()),inplace=True)
    df.drop(["support","recall","precision"],inplace=True)
    for key,val in params.items():
        df[key] = val
    df = df.set_index("dataset")
    big_df = big_df.append(df)
print(big_df)
big_df.to_csv(csv_folder + "_test_results.csv")

             AnnualCrop    Forest  HerbaceousVegetation   Highway  Industrial  \
dataset                                                                         
eurosat_rgb    0.694656  0.886667              0.632258  0.511156    0.869888   
eurosat_rgb    0.906149  0.979798              0.883721  0.898551    0.954274   
eurosat_rgb    0.649299  0.783005              0.416988  0.434783    0.797619   
eurosat_rgb    0.897690  0.974874              0.880000  0.833645    0.936255   
eurosat_rgb    0.850534  0.970297              0.858086  0.808511    0.923664   

              Pasture  PermanentCrop  Residential     River   SeaLake  \
dataset                                                                 
eurosat_rgb  0.653928       0.517454     0.881686  0.514403  0.843972   
eurosat_rgb  0.912195       0.857724     0.976667  0.858351  0.965636   
eurosat_rgb  0.456576       0.494983     0.769772  0.526902  0.721649   
eurosat_rgb  0.869565       0.839104     0.978369  0.804545  0.9363